In [1]:
!git clone https://github.com/WorkInTheDark/FairytaleQA_Baseline.git
%cd /content/FairytaleQA_Baseline/

Cloning into 'FairytaleQA_Baseline'...
remote: Enumerating objects: 1842, done.
remote: Counting objects: 100% (1842/1842), done.
remote: Compressing objects: 100% (995/995), done.
remote: Total 1842 (delta 304), reused 1833 (delta 302), pack-reused 0
Receiving objects: 100% (1842/1842), 6.96 MiB | 21.47 MiB/s, done.
Resolving deltas: 100% (304/304), done.


download model to ```models```folder

In [ ]:
!mkdir /content/FairytaleQA_Baseline/models
MODEL_PATH = 

load libraries

In [3]:
%%capture
!pip install torch==1.8.0
!pip install urllib3==1.25.4
!pip install allennlp-models==1.2
!pip install spacy
!python -m spacy download en_core_web_sm

In [4]:
%%capture
!pip install future==0.17.1
!pip install pytorch torchvision cudatoolkit=10.2 -c pytorch
!pip install pytorch-lightning==0.8.5
!pip install -U torchtext==0.8.0

In [ ]:
#%%capture
%cd /contentFairytaleQA_Baseline
%cd ./transformers
!pip install -e ".[testing]"
!pip install -r ./examples/requirements.txt
%cd ./..


**Restart runtime here to complete installation**

In [6]:
print(a)

NameError: ignored

In [1]:
%cd /content/FairytaleQA_Baseline

/content/CQ_A_book_movie_summary_Mou


In [2]:

import argparse
import json
import time
import warnings
from logging import getLogger
from pathlib import Path
from typing import Dict, List
import argparse
import glob
import logging
import os
import time
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Tuple
import numpy as np
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from lightning_base import BaseTransformer, add_generic_args, generic_train
from transformers import MBartTokenizer, T5ForConditionalGeneration
from transformers.modeling_bart import shift_tokens_right

try:
    from .callbacks import Seq2SeqLoggingCallback, get_checkpoint_callback, get_early_stopping_callback
    from .utils import (
        ROUGE_KEYS,
        LegacySeq2SeqDataset,
        Seq2SeqDataset,
        assert_all_frozen,
        calculate_bleu,
        calculate_rouge,
        flatten_list,
        freeze_params,
        get_git_info,
        label_smoothed_nll_loss,
        lmap,
        pickle_save,
        save_git_info,
        save_json,
        use_task_specific_params,
        parse_numeric_cl_kwargs,
    )
except ImportError:
    from callbacks import Seq2SeqLoggingCallback, get_checkpoint_callback, get_early_stopping_callback
    from utils import (
        ROUGE_KEYS,
        LegacySeq2SeqDataset,
        Seq2SeqDataset,
        assert_all_frozen,
        calculate_bleu,
        calculate_rouge,
        flatten_list,
        freeze_params,
        get_git_info,
        label_smoothed_nll_loss,
        lmap,
        pickle_save,
        save_git_info,
        save_json,
        use_task_specific_params,
        parse_numeric_cl_kwargs,
    )

logger = logging.getLogger(__name__)

DEFAULT_DEVICE = "0" if torch.cuda.is_available() else "cpu"

class SummarizationModule(BaseTransformer):
    mode = "summarization"
    loss_names = ["loss"]
    metric_names = ROUGE_KEYS
    default_val_metric = "rouge2"

    def __init__(self, hparams, **kwargs):
        super().__init__(hparams, num_labels=None, mode=self.mode, **kwargs)
        use_task_specific_params(self.model, self.mode)
#         self.target_lens = {
#             "train": self.hparams.max_target_length,
#             "val": self.hparams.val_max_target_length,
#             "test": self.hparams.test_max_target_length,
#         }
#         self.decoder_start_token_id = None  # default to config
#         if self.model.config.decoder_start_token_id is None and isinstance(self.tokenizer, MBartTokenizer):
#             self.decoder_start_token_id = self.tokenizer.lang_code_to_id[hparams.tgt_lang]
#             self.model.config.decoder_start_token_id = self.decoder_start_token_id
#         self.test_max_target_length = self.hparams.test_max_target_length
   

    def freeze_embeds(self):
        """Freeze token embeddings and positional embeddings for bart, just token embeddings for t5."""
        try:
            freeze_params(self.model.model.shared)
            for d in [self.model.model.encoder, self.model.model.decoder]:
                freeze_params(d.embed_positions)
                freeze_params(d.embed_tokens)
        except AttributeError:
            freeze_params(self.model.shared)
            for d in [self.model.encoder, self.model.decoder]:
                freeze_params(d.embed_tokens)

    def forward(self, input_ids, **kwargs):
        return self.model(input_ids, **kwargs)

    def ids_to_clean_text(self, generated_ids: List[int]):
        gen_text = self.tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        return lmap(str.strip, gen_text)


    def generate(self, input_ids, attention_mask, **generate_kwargs):
        # pad_token_id = self.tokenizer.pad_token_id
        # source_ids, source_mask, y = SummarizationDataset.trim_seq2seq_batch(batch, pad_token_id)
        generated_ids = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            num_beams=1,
            max_length=20,
            min_length=1,
            repetition_penalty=1.5,
            length_penalty=3,
            early_stopping=True,
            use_cache=False,
            **generate_kwargs
        )
        return generated_ids

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]


def generate_summaries_or_translations(
    examples: List[str],
    model_name: str,
    batch_size: int = 8,
    device: str = DEFAULT_DEVICE,
    fp16=False,
    task="summarization",
    prefix='',
    args=None,
    **generate_kwargs,
) -> Dict:
    # save outputs to a list
    output_list = []


    ## 0421 -- load model outside
    '''
    model_name = str(model_name)

    if "summarization" in args.task:
        model: SummarizationModule = SummarizationModule(args)
    else:
        model: SummarizationModule = TranslationModule(args)
    model = model.load_from_checkpoint(args.ckpt_path)
    model.eval()
    
    if args.device != 'cpu':
      model.to('cuda:{}'.format(args.device))
    '''
    
    # tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=args.cache_dir)
    # tokenizer.add_special_tokens({'sep_token': '<SEP>'})
    # model.resize_token_embeddings(len(tokenizer))
    
    # print('#############################################')
    # print("# model is loaded from", args.ckpt_path)
    # print('# tokenizer.all_special_tokens =', model.tokenizer.all_special_tokens)
    # print('# tokenizer.all_special_ids =', model.tokenizer.all_special_ids)
    # print('#############################################')
    

    # update config with task specific params
    use_task_specific_params(model, task)
    for examples_chunk in tqdm(list(chunks(examples, batch_size))):
        examples_chunk = [prefix + text for text in examples_chunk]
        
        if device=='cpu':
          batch = model.tokenizer(examples_chunk, return_tensors="pt", truncation=True, padding="longest")
        else:
          batch = model.tokenizer(examples_chunk, return_tensors="pt", truncation=True, padding="longest").to('cuda:{}'.format(device))
        
        ###batch = model.tokenizer(examples_chunk, return_tensors="pt", truncation=True, padding="longest")
        
        if len(batch.input_ids[0]) > 1024:
            
            if device=='cpu':
              end_token = model.tokenizer.encode('</s>', return_tensors='pt')[0][-2:]
            else:
              end_token = model.tokenizer.encode('</s>', return_tensors='pt')[0][-2:].to('cuda:{}'.format(device))
            
            ###end_token = model.tokenizer.encode('</s>', return_tensors='pt')[0][-2:]
            
            input_ids = torch.cat((batch.input_ids[0][:1022], end_token), 0).unsqueeze(0)
            batch.input_ids = input_ids
        # batch.input_ids[0,-1] = 2
        # print(batch.input_ids)
        # print(batch.attention_mask)
        # return
        summaries = model.generate(
            input_ids=batch.input_ids,
            attention_mask=batch.attention_mask,
            **generate_kwargs,
        )
        dec = model.tokenizer.batch_decode(summaries, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        for hypothesis in dec:
            output_list.append(hypothesis)

    return output_list


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("--model_name", type=str, help="like facebook/bart-large-cnn,t5-base, etc.")
parser.add_argument("--input_path", type=str, help="like cnn_dm/test.source")
parser.add_argument("--save_path", type=str, help="where to save summaries")
parser.add_argument("--reference_path", type=str, required=False, help="like cnn_dm/test.target")
parser.add_argument("--score_path", type=str, required=False, default="metrics.json", help="where to save metrics")
parser.add_argument("--device", type=str, required=False, default=DEFAULT_DEVICE, help="cuda, cuda:1, cpu etc.")
parser.add_argument(
    "--prefix", type=str, required=False, default='', help="will be added to the begininng of src examples"
)
parser.add_argument("--task", type=str, default="summarization", help="used for task_specific_params + metrics")
parser.add_argument("--bs", type=int, default=1, required=False, help="batch size")
parser.add_argument(
    "--n_obs", type=int, default=-1, required=False, help="How many observations. Defaults to all."
)
parser.add_argument("--fp16", action="store_true")
################################
parser.add_argument(
        "--cache_dir",
        default="./pretrained/",
        type=str,
        help="Where do you want to store the pre-trained models downloaded from s3",
    )
parser.add_argument(
        "--ckpt_path",
        type=str,
        help='path tooo stored model checkpoints',
    )
parser.add_argument("--output_dir", default="./", type=str)
parser.add_argument("--model_name_or_path", type=str, default="facebook/bart-large", help="like facebook/bart-large-cnn,t5-base, etc.")
parser.add_argument("--config_name", type=str, default="facebook/bart-large")
parser.add_argument("--tokenizer_name", type=str, default="facebook/bart-large")
parser.add_argument("--test_max_target_length", type=int)
parser.add_argument("--eval_max_length", type=int)
################################
# Unspecified args like --num_beams=2 --decoder_start_token_id=4 are passed to model.generate
args, rest = parser.parse_known_args()

In [4]:
# import torch

model: SummarizationModule = SummarizationModule(args)

model = model.load_from_checkpoint(MODEL_PATH)


model.eval()

DEFAULT_DEVICE = "0" if torch.cuda.is_available() else "cpu"

if DEFAULT_DEVICE != 'cpu':
  model.to('cuda:{}'.format(DEFAULT_DEVICE))


print('#############################################')
print("# model is loaded from", args.ckpt_path)
print('# tokenizer.all_special_tokens =', model.tokenizer.all_special_tokens)
print('# tokenizer.all_special_ids =', model.tokenizer.all_special_ids)
print('#############################################')

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['encoder.embed_events.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['encoder.embed_events.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#############################################
# model is loaded from None
# tokenizer.all_special_tokens = ['<s>', '</s>', '<unk>', '<SEP>', '<pad>', '<s>', '<mask>', '<RES>']
# tokenizer.all_special_ids = [0, 2, 3, 50265, 1, 0, 50264, 50266]
#############################################


In [ ]:
def run_generate(examples):
  parsed = {}
  generate_results = generate_summaries_or_translations(
      examples,
      args.model_name_or_path,
      batch_size=args.bs,
      device=args.device,
      fp16=args.fp16,
      task=args.task,
      prefix=args.prefix,
      args=args,
      **parsed,
  )

  return generate_results

### For QA Task

In [ ]:
def e2eQA(input_content, dataset_type):
  input_for_QA_model = []
  QC_lines = open(input_content, 'r').readlines()
  for i in QC_lines:
    input_for_QA_model.append( i.replace('\n', '') )

  output_for_QA_model = run_generate(input_for_QA_model)
  for i in range(len(output_for_QA_model)):
    output_for_QA_model[i] = output_for_QA_model[i].replace("\n", "").split(".")[0].strip(" .") + " ."

  assert len(input_for_QA_model) == len(output_for_QA_model)


  f = open("/BART_QA/results/predict_" + dataset_type + ".txt", 'w')

  for i in range(len(output_for_QA_model)):
    f.write( output_for_QA_model[i] + '\n')
  
  f.close()


  return []


In [ ]:
import glob 

if __name__ == "__main__":

  qa_pair_list = e2eQA('/BART_QA/test.source', 'test') 
  qa_pair_list = e2eQA('/BART_QA/val.source', 'val') 

 

### For QG Task

In [ ]:
def e2eQG(input_content, dataset_type):
  input_for_QG_model = []
  QC_lines = open(input_content, 'r').readlines()
  for i in QC_lines:
    input_for_QG_model.append( i.replace('\n', '') )

  output_for_QG_model = run_generate(input_for_QG_model)
  for i in range(len(output_for_QG_model)):
    output_for_QG_model[i] = output_for_QG_model[i].replace("\n", "").split("?")[0].strip(" ?") + " ?"

  
  # print(output)

  assert len(input_for_QG_model) == len(output_for_QG_model)

  f = open("/BART_QG/results/predict_" + dataset_type + ".txt", 'w')

  for i in range(len(output_for_QG_model)):
    f.write( output_for_QG_model[i] + '\n')
  
  f.close()


  return []


In [ ]:
import glob 

if __name__ == "__main__":

  qa_pair_list = e2eQG('/BART_QG/test.source', 'test') 
  qa_pair_list = e2eQG('/BART_QG/val.source', 'val') 

 